In [1]:
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

torch:  1.10 ; cuda:  1.10.0


In [2]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [3]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("road_defect_detection_train", {}, "cropped_images/train_depth.json", "cropped_images/train_depth")
register_coco_instances("road_defect_detection_val", {}, "cropped_images/depth_09/annotations/instances_default.json", "cropped_images/depth_09/images")

my_dataset_train_metadata = MetadataCatalog.get("road_defect_detection_train")
dataset_dicts = DatasetCatalog.get("road_defect_detection_train")

[05/28 16:21:19 d2.data.datasets.coco]: Loaded 13 images in COCO format from cropped_images/train_depth.json


In [4]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.WEIGHTS = 'depth_model_0000749.pth'
cfg.DATASETS.TRAIN = ("road_defect_detection_train",)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

In [6]:
from detectron2.utils.visualizer import ColorMode
# import matplotlib.pyplot as plt
test_path = "cropped_images/depth_09/images"

for filename in os.listdir(test_path):
    f = os.path.join(test_path, filename)
    print(f)
    im = cv2.imread(f)
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    out_filename = filename[:-4] + "_out.png"
    cv2.imwrite(out_filename, v.get_image()[:, :, ::-1])

cropped_images/depth_09/images\tile_09_depth_0_0.png
cropped_images/depth_09/images\tile_09_depth_0_1600.png
cropped_images/depth_09/images\tile_09_depth_0_2400.png
cropped_images/depth_09/images\tile_09_depth_0_3200.png
cropped_images/depth_09/images\tile_09_depth_0_4000.png
cropped_images/depth_09/images\tile_09_depth_0_5600.png
cropped_images/depth_09/images\tile_09_depth_0_6400.png
cropped_images/depth_09/images\tile_09_depth_0_7200.png
cropped_images/depth_09/images\tile_09_depth_0_800.png
cropped_images/depth_09/images\tile_09_depth_0_8000.png
cropped_images/depth_09/images\tile_09_depth_0_8800.png


In [5]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("road_defect_detection_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "road_defect_detection_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

[05/28 16:23:35 d2.data.datasets.coco]: Loaded 11 images in COCO format from cropped_images/depth_09/annotations/instances_default.json
[05/28 16:23:36 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|  Potholes  | 49           |
|            |              |
[05/28 16:23:36 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[05/28 16:23:36 d2.data.common]: Serializing 11 elements to byte tensors and concatenating them all ...
[05/28 16:23:36 d2.data.common]: Serialized dataset takes 0.01 MiB
[05/28 16:23:36 d2.evaluation.evaluator]: Start inference on 11 batches


C:\Users\luope\Anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/28 16:23:51 d2.evaluation.evaluator]: Inference done 1/11. Dataloading: 9.5706 s/iter. Inference: 5.7427 s/iter. Eval: 0.0001 s/iter. Total: 15.3139 s/iter. ETA=0:02:33
[05/28 16:23:53 d2.evaluation.evaluator]: Total inference time: 0:00:01.693689 (0.282281 s / iter per device, on 1 devices)
[05/28 16:23:53 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:01 (0.223845 s / iter per device, on 1 devices)
[05/28 16:23:53 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/28 16:23:53 d2.evaluation.coco_evaluation]: Saving results to ./output\coco_instances_results.json
[05/28 16:23:53 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[05/28 16:23:53 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[05/28 16:23:53 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.00 seconds.
[05/28 16:23:53 d2.evaluation.